<a href="https://colab.research.google.com/github/Cosmetia/IA_formula/blob/main/CostmetIA_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation du modèle de langage dans l'environnement

In [ ]:
!pip install crewai 'crewai[tools]'
!pip install langchain-core
!pip install langchain-chroma
!pip install langchain-community
!pip install install chromadb

ERROR: Could not find a version that satisfies the requirement install (from versions: none)
ERROR: No matching distribution found for install


## Initialisation des librairies et stockage

In [ ]:
#import ollama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFDirectoryLoader
import os
from langchain_openai import ChatOpenAI
from crewai import Agent
from crewai_tools import PDFSearchTool
import json
from langchain_community.llms import Ollama
import dotenv
from langchain_community.document_loaders.merge import MergedDataLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain_community.callbacks import get_openai_callback

Gdrive contenant la base des formules

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
%cd /content/drive/MyDrive/cosmetia


Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1n0WWZdbu8CfY2VqjoUBWHQ27zkbaKyoI/cosmetia


In [ ]:
dotenv.load_dotenv()

True

## Initialisation de la base de donnée

In [ ]:
## Initialisation de la base de donnée de formules
pdf_loader = PyPDFDirectoryLoader('for/')
docs = pdf_loader.load()
pdf_splits=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100).split_documents(docs)
formulas = Chroma.from_documents(documents=pdf_splits, embedding=OpenAIEmbeddings(),persist_directory="./chromadb1")

AuthenticationError: Error code: 401 - {'error': {'message': 'You do not have access to the organization tied to the API key.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_organization'}}

In [ ]:
## Initialisation de la base de donnée de livres cosmetique
pdf_loader = PyPDFDirectoryLoader('livres/')
books = pdf_loader.load()
splits=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100).split_documents(books)
books=Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(),persist_directory="./chromadb2")

In [ ]:

## Initialisation de la base de donnée de matière première et liste inci
mp_loader=UnstructuredExcelLoader(file_path="MP/MP.xlsx")
inci_loader=CSVLoader(file_path="inci_list/ZXW.csv")
loader_mp=MergedDataLoader(loaders=[mp_loader,inci_loader])
ingredients_csv=loader_mp.load()
splits=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100).split_documents(ingredients_csv)
ingredients=Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(),persist_directory="./chromadb3")


In [ ]:
## Initialisation de la base de donnée des régulations
loader = PyPDFDirectoryLoader('Regulation/')
docs = loader.load()
splits=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100).split_documents(docs)
regulation=Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(),persist_directory="./chromadb4")

Initialisation des modèles de langages

In [ ]:
gpt4=ChatOpenAI(model='gpt-4-turbo-preview',temperature=0,streaming=False)
gpt3_5=ChatOpenAI(model='gpt-3.5-turbo',temperature=0,streaming=False)


In [ ]:
from crewai import Agent,Task, Crew, Process
from langchain.tools import tool

## Déclarations des outils que les agents pourront utiliser

In [ ]:
## Outils d'extraction d'informations sur des formules cosmetiques

class Formula:
  @tool("Get Formula Tool")
  def formulaDB(query: str)-> str:
      """ Search in cosmetic product database for relevant cosmetic product formula based on a query"""
      vectorStore=Chroma(persist_directory="./chromadb1",embedding_function=OpenAIEmbeddings())
      retriever = vectorStore.similarity_search(query)
      return retriever



In [ ]:
## Outil d'extraction de règles de formulation à partir des livres
class Livres:
  @tool("Get Formulation rules Tool")
  def booksDB(query: str)-> str:
      """ Search in cosmetic books database for relevant product formulation rules based on a query"""
      vectorStore=Chroma(persist_directory="./chromadb2",embedding_function=OpenAIEmbeddings())
      retriever = vectorStore.similarity_search(query)
      return retriever

In [ ]:
# Outil d'extraction des noms inci et leurs propriétés
class Ingredients:
  @tool("Get Ingredients info Tool")
  def ingredientsDB(query: str)-> str:
      """ Search in ingredients and inci names database for relevant ingredients and properties based on a query"""
      vectorStore=Chroma(persist_directory="./chromadb3",embedding_function=OpenAIEmbeddings())
      retriever = vectorStore.similarity_search(query)
      return retriever

In [ ]:
# Outil d'extraction des régulations sur les produits cosmetiques
class Regulation:
  @tool("Get Regulation Tool")
  def regulationDB(query: str)-> str:
      """ Search in the regulation document database for relevant regulation based on a query"""
      vectorStore=Chroma(persist_directory="./chromadb4",embedding_function=OpenAIEmbeddings())
      retriever = vectorStore.similarity_search(query)
      return retriever

In [ ]:
llm=gpt3_5

Création des agents IA

In [ ]:
input_analyst = Agent(
    role="Product Requirements Analyzer",
    goal="Interpret user input and extract key product features :",
    backstory="You're an expert in cosmetic product, understanding consumer needs and translating them into technical requirements for cosmetic formulator.",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

formula_researcher = Agent(
    role="Cosmetic Formulation Specialist",
    goal="Identify suitable base formulas from existing formulations",
    backstory="With years of experience in cosmetic chemistry, you excel at finding and adapting formulas to meet specific requirements.",
    verbose=True,
    allow_delegation=False,
    tools=[Formula.formulaDB],
    llm=llm
)

ingredient_analyst = Agent(
    role="Ingredient Expert",
    goal="Match product requirements received with suitable INCI ingredients. Use the Get Ingredients info Tool to look for information about INCI name property. Use Get formula tool to search for an INCI names in proprietary cosmetic formula database. ",
    backstory="Your deep knowledge of cosmetic ingredients allows you to select the perfect components for any formulation.",
    verbose=True,
    allow_delegation=False,
    tools=[Ingredients.ingredientsDB,Formula.formulaDB],
    llm=llm
)

regulatory_specialist = Agent(
    role="Cosmetic Regulatory Specialist",
    goal="Ensure formula compliance with relevant regulations. Use the Get Regulation tool to look for relevant cosmetic regulations in our database. ",
    backstory="Your expertise in global cosmetic regulations keeps products safe and legally compliant across markets.",
    verbose=True,
    allow_delegation=False,
    tools=[Regulation.regulationDB],  # Assuming regulatory info is in the cosmetic books
    llm=llm
)

formula_synthesizer = Agent(
    role="Senior Cosmetic Formulator",
    goal="Create a complete, balanced formula composed of INCI names and their proportions based on all inputs. ",
    backstory="You're a master at bringing together diverse inputs to create innovative, effective cosmetic formulas.",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

quality_controller = Agent(
    role="Cosmetic Quality Assurance Specialist",
    goal="Evaluate and refine the proposed formula",
    backstory="Your attention to detail and broad knowledge base ensure that every formula meets the highest standards of quality and performance.",
    verbose=True,
    allow_delegation=False,
    tools=[Livres.booksDB, Ingredients.ingredientsDB],
    llm=llm
)
multipurpose_scientist = Agent(
    role="Mathematician and chemist senior",
    goal="Total the quatinted ingredients and make sure it gives 100%, change one ingredient for another that is more appropriate with an adequate rate.",
    backstory="You are very good at calculation and reasoning, you look at all the cosmetic formulas before releasing the final version.",
    verbose=True,
    allow_delegation=False,
    tools=[Livres.booksDB, Ingredients.ingredientsDB, Regulation.regulationDB, Formula.formulaDB],
    llm=gpt4
)
product_manager = Agent(
  role='Cosmetic Product Manager',
  goal='Ensure the smooth operation and coordination of the cosmetic formulation process',
  verbose=True,
  backstory=(
    """Senior cosmetic product manager at a renowed cosmetic lab. As a seasoned project manager, you excel in organizing
    tasks, and ensuring the team stays on track. Deliver tailored specific product formulation with complete (100%) ingredient quantity."""
  ),
  allow_code_execution=True,
  llm=gpt4
)

Tâches

In [ ]:
# Create tasks
input_analysis = Task(
    description="""Analyze the user's product description and extract a tailored cosmetic product requirements For Formulation Purpose.
    Input : {description}
    """,
    agent=input_analyst,
    expected_output="A structured list of cosmetic product requirements."
)

formula_search = Task(
    description="""Research and identify suitable base formulas from existing cosmetic formulations.
    You must think step by step and use the following tools accordingly :
    - Use Get Formula Tool for searching similar product formula in the database
    - Use Get Formulation rules Tool for searching relevant information from cosmetic books
    - Use Get Ingredients info Tool for searching raw ingredients INCI names and properties
    Steps :
    - Search in database for cosmetic formulas based on product requirements
    - Identify and extract relevant INCI ingedients formula that match the desired product type
    - Analyze common INCI ingredients and their proportions in similar formulas
    - Compile a list of potential base formulas with rationale for each selection

    Constraint :  mention only the INCI names and not the raw ingredient name or any proprietary trademarked cosmetic ingredients
    """,
    agent=formula_researcher,
    # output des autres taches à ajouter comme contexte
    context=[input_analysis],
    tools=[Livres.booksDB, Formula.formulaDB,Ingredients.ingredientsDB],
    expected_output="A list of potential base formulas with rationale for each selection."
)

ingredient_analysis = Task(
    description="""Analyze and recommend specific ingredients based on product requirements. Use the Get Ingredients tool to search in the raw material and inci list.Think step by step.
        -Match ingredients to desired properties and product requirements
        -Analyze ingredient compatibility and stability
        -Provide a detailed report on each recommended ingredient, including INCI name and function
    """,
    agent=ingredient_analyst,
    # output des autres taches à ajouter comme contexte7
    context=[input_analysis],
    tools=[Ingredients.ingredientsDB],
    expected_output="Detailed report INCI ingredient names and function tailored to the product"
)



formula_synthesis = Task(
    description="""Synthesize all inputs to create a complete, balanced formula. Think step by step.
    Do not forget that the total percentage of all the ingredients must be 100%.
    """,
    agent=formula_synthesizer,
    # output des autres taches à ajouter comme contexte
    context=[input_analysis,ingredient_analysis,formula_search],
    expected_output="A balanced and detailed cosmetic formulation"
)

regulatory_compliance = Task(
    description="""Ensure regulatory compliance of the proposed formula, referencing relevant regulations. If there is no regulation found, skip this step
    """,
    agent=regulatory_specialist,
    # output des autres taches à ajouter comme contexte
    context=[input_analysis,formula_synthesis],
    tools=[Regulation.regulationDB],
    expected_output="A compliance report and suggestions for any necessary adjustments"
)

quality_control = Task(
    description="""Evaluate the proposed formula for quality.
    Steps : 1- replace every single ingredients with its inci name  in the ingredients database and books, if there's no match, leave it as it is.
            2- Calculate step by step the total quantity percentage of each inci names in the formula by summing them up
            3- If the total quantity percentage is different than 100% the formula is not balanced and should be refined by the formula synthesizer agent

    """,
    agent=quality_controller,
    # output des autres taches à ajouter comme contexte
    context=[formula_synthesis],
    tools=[Livres.booksDB, Ingredients.ingredientsDB],
    expected_output="A list of potential refinements or alternatives to improve the formula"
)

before_formula=Task(
    description="""Evaluate the proposed formula for quality and quantity.
    Steps : 1- replace one ingredient by another that is more appropriate with an adequate rate.
            2- Calculate step by step the total quantity percentage of each inci names in the formula by summing them up
            3- If the total quantity percentage is different than 100%, try to give each ingredient a more appropriate rate and try again.
            4- Make sure that the formula meets the customer's needs and the total quantity percentage is 100%.

    """,
    agent=multipurpose_scientist,
    context=[formula_synthesis,regulatory_compliance,quality_control],
    tools=[Livres.booksDB, Ingredients.ingredientsDB, Regulation.regulationDB, Formula.formulaDB],
    expected_output="A list of potential refinements or alternatives to improve the formula with appropiate quantity"
)


final_formula=Task(
    description="""Synthesize all inputs to create a final formula that meets the product requirements.
    Initial user description : {description}
    Steps :
    -Synthesize context inputs from all other agents
    -Develop a complete formula with precise ingredient percentages and preparation phase, using only INCI names.
    -Replace One By One The Ingredients Name With Their Inci Name, Use the the Get Ingredient tool to match ingredient with inci names From Database, If There Is No Result Leave it as it is if there i that match.
    -Adjust the formula for optimal stability, efficacy, and sensory properties
    -Provide a detailed formulation tailored to the product requirement
    Make sure that the formula contain only INCI names, use the the Get Ingredient tool to match ingredient with inci names. Leave it as it is if there is no inci names that match in the data """,
    agent=formula_synthesizer,
    # output des autres taches à ajouter comme contexte
    context=[formula_synthesis,regulatory_compliance,quality_control],
    tools=[Ingredients.ingredientsDB],
    expected_output="A JSON object with the complete formulation protocol with INCI names , Qty % , functions and the preparation procedures "
)

Création du crew

In [ ]:
cosmetic_crew=Crew(
    agents=[input_analyst,formula_researcher,ingredient_analyst,regulatory_specialist,formula_synthesizer,quality_controller, multipurpose_scientist],
    tasks=[input_analysis,formula_search,ingredient_analysis,formula_synthesis,regulatory_compliance,quality_control, before_formula, final_formula],
    manager_agent=product_manager,
    process=Process.hierarchical,output_json=True
)

Cette valeur doit être zéro sinon CrewAI bug ( relancer l'initialisation des agents corrige l'érreur )

In [ ]:
len(product_manager.tools)

0

#Lancement du crew et génération de la formule

In [ ]:
inp={"description":"une crème hydratante aloe vera"}

In [ ]:
with get_openai_callback() as cb:
  formula=cosmetic_crew.kickoff(inputs=inp)
  print(cb)



> Entering new CrewAgentExecutor chain...
I need to gather detailed information on the specific requirements for formulating an aloe vera moisturizing cream to ensure the formulation process is smooth and efficient. This includes understanding the desired properties, target audience, and any regulatory considerations that might affect the formulation. Since I don't have direct access to all the necessary information, I will need to delegate this task to the Product Requirements Analyzer, who can help me compile a comprehensive list of product requirements.

Action: Delegate work to coworker

Action Input: {"coworker": "Product Requirements Analyzer", "task": "Analyze product description and extract detailed requirements for formulation", "context": "We are tasked with formulating an aloe vera moisturizing cream. This product is aimed at consumers looking for a hydrating skincare solution that incorporates the soothing properties of aloe vera. The formulation should be suitable for al

In [ ]:
 print(formula)

```json
{
  "formula": [
    {
      "INCI_name": "Sodium Hyaluronate",
      "Qty_%": 1,
      "Function": "Hydration and plumping effect"
    },
    {
      "INCI_name": "Mentha Viridis Leaf Oil",
      "Qty_%": 0.5,
      "Function": "Tingling sensation for temporary plumping effect"
    },
    {
      "INCI_name": "Cinnamomum Zeylanicum Extract",
      "Qty_%": 0.5,
      "Function": "Stimulates blood flow for plumping effect"
    },
    {
      "INCI_name": "CI 15850",
      "Qty_%": 10,
      "Function": "Vibrant red color"
    },
    {
      "INCI_name": "Butyrospermum Parkii Butter",
      "Qty_%": 15,
      "Function": "Moisturizes and softens lips"
    },
    {
      "INCI_name": "Simmondsia Chinensis Seed Oil",
      "Qty_%": 10,
      "Function": "Emollient for smooth application"
    },
    {
      "INCI_name": "Vegetable Oil",
      "Qty_%": 12,
      "Function": "Base and emollient"
    },
    {
      "INCI_name": "Euphorbia Cerifera Cera",
      "Qty_%": 15,
      "Func

In [ ]:
  print(formula)

```json
{
  "formula": [
    {"INCI_name": "Ricinus Communis (Castor) Seed Oil", "Qty %": 20, "Function": "Emollient", "Preparation": "Melt together with waxes"},
    {"INCI_name": "Cera Alba (Beeswax)", "Qty %": 15, "Function": "Thickener/Emulsifier", "Preparation": "Melt"},
    {"INCI_name": "Copernicia Cerifera (Carnauba) Wax", "Qty %": 5, "Function": "Thickener/Texture enhancer", "Preparation": "Melt"},
    {"INCI_name": "Butyrospermum Parkii (Shea) Butter", "Qty %": 10, "Function": "Emollient", "Preparation": "Melt"},
    {"INCI_name": "Lanolin", "Qty %": 5, "Function": "Emollient", "Preparation": "Melt"},
    {"INCI_name": "Euphorbia Cerifera (Candelilla) Wax", "Qty %": 5, "Function": "Thickener/Texture enhancer", "Preparation": "Melt"},
    {"INCI_name": "Ozokerite", "Qty %": 5, "Function": "Thickener", "Preparation": "Melt"},
    {"INCI_name": "Isododecane", "Qty %": 10, "Function": "Solvent/Texture enhancer", "Preparation": "Mix with silicones"},
    {"INCI_name": "Dimethicone